<p style = 'color:#E6E4E7 ; background-color:#00561B ; text-align:center ; font-size: 300%'> Kayak project - Get hotel data for Lille 

<p style = 'color:#00561B ; background-color:#E6E4E7 ; text-align:center ; font-size: 150%'> Project prepared by Stephanie Cotineau - #dsmpt-Paris-08

In [1]:
!pip install Scrapy -q

In [2]:
import os # For manipulate file 
import logging # For display logs
import scrapy # Import scrapy & scrapy.crawler
from scrapy.crawler import CrawlerProcess
import re

In [3]:
# Name of the file where the results will be saved
filename = "Lille_multipage.json"

# If file already exists, delete it before crawling (because Scrapy will 
# concatenate the last and new results otherwise)
if filename in os.listdir('results_hotels/'):
        os.remove('results_hotels/' + filename)

# Declare a new CrawlerProcess with some settings
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'
process = CrawlerProcess(settings = {
    'USER_AGENT': user_agent,
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": {
        'results_hotels/' + filename : {"format": "json"},
    }
})

In [4]:
from bs4 import BeautifulSoup 
import requests

In [5]:
# Lille - 5 first pages
offset = 0
link_offset = []
while offset != 125:
    url = f"https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=lille&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=lille&search_pageview_id=15c43bf6873c02b4&offset={offset}"
    #print(url)
    offset = offset + 25
    link_offset.append(url)

In [6]:
link_offset

['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=lille&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=lille&search_pageview_id=15c43bf6873c02b4&offset=0',
 'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index

In [7]:
class LilleSpider(scrapy.Spider) :
    # Name of spider
    name = 'Lille'
    
    # Starting URL
    start_urls = ['https://www.booking.com/']
    
    def start_requests(self):
        for d in ['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=lille&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=lille&search_pageview_id=15c43bf6873c02b4&offset=0',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=lille&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=lille&search_pageview_id=15c43bf6873c02b4&offset=25',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=lille&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=lille&search_pageview_id=15c43bf6873c02b4&offset=50',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=lille&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=lille&search_pageview_id=15c43bf6873c02b4&offset=75',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=lille&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=lille&search_pageview_id=15c43bf6873c02b4&offset=100']:
            yield scrapy.Request(d,callback=self.search)
    
    # Callback used after search launched
    def search(self, response):
        hotels = response.css('div.a826ba81c4.fe821aea6c.fa2f36ad22.afd256fc79.d08f526e0d.ed11e24d01.da89aeb942')
        for hotel in hotels:
            yield {
                'location' : hotel.css('span.f4bd0794db.b4273d69aa::text').get(),
                'hotel_url' : hotel.css('a.e13098a59f').attrib['href'],
                'hotel_name' : hotel.css('div.fcab3ed991.a23c043802::text').get(),
                'hotel_rating' : hotel.css('div.b5cd09854e.d10a6220b4::text').get(),
                'hotel_description_short' : hotel.css('div.d8eab2cf7f::text').get()
            }

In [8]:
# Start the crawling using the spider defined above
process.crawl(LilleSpider)
process.start()

In [9]:
import pandas as pd

In [10]:
lille = pd.read_json('results_hotels/Lille_multipage.json')

In [11]:
len(lille)

125

In [12]:
lille.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short
0,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/moxy-lille-ci...,Moxy Lille City,"8,6","Le Moxy Lille City est situé à Lille, à 700 mè..."
1,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/lille-europe....,Hotel Lille Europe,"8,0",L'Hotel Lille Europe est un établissement 3 ét...
2,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/calm-lille.fr...,Hôtel Calm Lille,"7,7",L'Hôtel Calm Lille est situé dans le centre-vi...
3,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/lille-avenue....,Lille Avenue,"8,1",Hébergement géré par un particulier
4,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/expressbyholi...,"Holiday Inn Express Lille Centre, an IHG Hotel","8,0",L'Holiday Inn Express Lille Centre est situé d...
5,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/ibis-styles-l...,ibis Styles Lille Centre Gare Beffroi,"7,8",L’ibis Styles Lille Centre Gare Beffroi vous a...
6,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/residence-lil...,Séjours & Affaires Lille Europe,"7,3",La résidence Séjours & Affaires Lille Europe e...
7,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/calm-appartho...,CALM Appart' & Hostel,"8,1","Le Calm Apparthotel vous accueille à Lille, à ..."
8,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/grandhotelbel...,Grand Hotel Bellevue - Grand Place,"8,5","Situé sur la Grand-Place, dans le Vieux-Lille,..."
9,"Fives, Lille",https://www.booking.com/hotel/fr/eklo-hotels-l...,Eklo Hotels Lille,"7,7","Doté d'une terrasse, l'Eklo Hotels Lille est s..."


In [13]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_ll =[]
for link in lille['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    lat_lon = soup_link.find_all(id = "hotel_address")
    for ll in lat_lon : 
        #print(ll['data-atlas-latlng'])
        result_ll.append(ll['data-atlas-latlng'])

In [14]:
len(result_ll)

125

In [15]:
lille['Latitude_Longitude'] = result_ll

In [16]:
lat = []
lon = []

# For each row in a varible,
for row in lille['Latitude_Longitude']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
lille['latitude'] = lat
lille['longitude'] = lon

In [17]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_long_description =[]
for link in lille['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    long_description = soup_link.find_all(id = "property_description_content") 
    for ld in long_description : 
        result_long_description.append(ld.text)

In [18]:
len(result_long_description)

125

In [19]:
lille['hotel_description_long'] = result_long_description

In [20]:
lille.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/moxy-lille-ci...,Moxy Lille City,"8,6","Le Moxy Lille City est situé à Lille, à 700 mè...","50.62783100,3.06359200",50.62783100,3.06359200,"\nLe Moxy Lille City est situé à Lille, à 700 ..."
1,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/lille-europe....,Hotel Lille Europe,"8,0",L'Hotel Lille Europe est un établissement 3 ét...,"50.63788756,3.07268500",50.63788756,3.07268500,\nVous pouvez bénéficier d'une réduction Geniu...
2,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/calm-lille.fr...,Hôtel Calm Lille,"7,7",L'Hôtel Calm Lille est situé dans le centre-vi...,"50.63528798,3.06970725",50.63528798,3.06970725,\nVous pouvez bénéficier d'une réduction Geniu...
3,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/lille-avenue....,Lille Avenue,"8,1",Hébergement géré par un particulier,"50.63340664,3.06223664",50.63340664,3.06223664,\nLe Lille Avenue bénéficie d'un emplacement c...
4,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/expressbyholi...,"Holiday Inn Express Lille Centre, an IHG Hotel","8,0",L'Holiday Inn Express Lille Centre est situé d...,"50.63075938,3.05808842",50.63075938,3.05808842,\nL'Holiday Inn Express Lille Centre est situé...
5,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/ibis-styles-l...,ibis Styles Lille Centre Gare Beffroi,"7,8",L’ibis Styles Lille Centre Gare Beffroi vous a...,"50.63238245,3.06794286",50.63238245,3.06794286,\nL’ibis Styles Lille Centre Gare Beffroi vous...
6,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/residence-lil...,Séjours & Affaires Lille Europe,"7,3",La résidence Séjours & Affaires Lille Europe e...,"50.63647023,3.07490923",50.63647023,3.07490923,\nLa résidence Séjours & Affaires Lille Europe...
7,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/calm-appartho...,CALM Appart' & Hostel,"8,1","Le Calm Apparthotel vous accueille à Lille, à ...","50.63734618,3.06945696",50.63734618,3.06945696,\nVous pouvez bénéficier d'une réduction Geniu...
8,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/grandhotelbel...,Grand Hotel Bellevue - Grand Place,"8,5","Situé sur la Grand-Place, dans le Vieux-Lille,...","50.63654707,3.06234241",50.63654707,3.06234241,\nVous pouvez bénéficier d'une réduction Geniu...
9,"Fives, Lille",https://www.booking.com/hotel/fr/eklo-hotels-l...,Eklo Hotels Lille,"7,7","Doté d'une terrasse, l'Eklo Hotels Lille est s...","50.62003296,3.08456697",50.62003296,3.08456697,"\nDoté d'une terrasse, l'Eklo Hotels Lille est..."


In [21]:
lille['hotel_description_long'] = lille['hotel_description_long'].replace('\n','', regex=True)

In [22]:
lille.head(5)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/moxy-lille-ci...,Moxy Lille City,"8,6","Le Moxy Lille City est situé à Lille, à 700 mè...","50.62783100,3.06359200",50.62783100,3.06359200,"Le Moxy Lille City est situé à Lille, à 700 mè..."
1,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/lille-europe....,Hotel Lille Europe,"8,0",L'Hotel Lille Europe est un établissement 3 ét...,"50.63788756,3.07268500",50.63788756,3.07268500,Vous pouvez bénéficier d'une réduction Genius ...
2,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/calm-lille.fr...,Hôtel Calm Lille,"7,7",L'Hôtel Calm Lille est situé dans le centre-vi...,"50.63528798,3.06970725",50.63528798,3.06970725,Vous pouvez bénéficier d'une réduction Genius ...
3,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/lille-avenue....,Lille Avenue,"8,1",Hébergement géré par un particulier,"50.63340664,3.06223664",50.63340664,3.06223664,Le Lille Avenue bénéficie d'un emplacement cen...
4,"Centre de Lille, Lille",https://www.booking.com/hotel/fr/expressbyholi...,"Holiday Inn Express Lille Centre, an IHG Hotel","8,0",L'Holiday Inn Express Lille Centre est situé d...,"50.63075938,3.05808842",50.63075938,3.05808842,L'Holiday Inn Express Lille Centre est situé d...


In [23]:
lille['hotel_description_long'][55]

"Situé dans le quartier de Moulins à Lille, à proximité de la maison Coilliot, le Superbe Appartement de standing au cœur de Lille dispose d'une connexion Wi-Fi gratuite et d'un lave-linge. Vous aurez accès à une cuisine entièrement équipée.Cet appartement comprend 4 chambres, 4 salles de bains et une télévision à écran plat.Vous séjournerez à proximité de l'Hospice Gantois, du Zénith de Lille et du Grand Palais de Lille. L'aéroport de Lille, le plus proche, est implanté à 6 km. "

In [29]:
# Clean location column
lille['location'] = ["Lille" if value!="Lille" else value for value in lille["location"]]

In [30]:
lille.head()

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Lille,https://www.booking.com/hotel/fr/moxy-lille-ci...,Moxy Lille City,"8,6","Le Moxy Lille City est situé à Lille, à 700 mè...","50.62783100,3.06359200",50.62783100,3.06359200,"Le Moxy Lille City est situé à Lille, à 700 mè..."
1,Lille,https://www.booking.com/hotel/fr/lille-europe....,Hotel Lille Europe,"8,0",L'Hotel Lille Europe est un établissement 3 ét...,"50.63788756,3.07268500",50.63788756,3.07268500,Vous pouvez bénéficier d'une réduction Genius ...
2,Lille,https://www.booking.com/hotel/fr/calm-lille.fr...,Hôtel Calm Lille,"7,7",L'Hôtel Calm Lille est situé dans le centre-vi...,"50.63528798,3.06970725",50.63528798,3.06970725,Vous pouvez bénéficier d'une réduction Genius ...
3,Lille,https://www.booking.com/hotel/fr/lille-avenue....,Lille Avenue,"8,1",Hébergement géré par un particulier,"50.63340664,3.06223664",50.63340664,3.06223664,Le Lille Avenue bénéficie d'un emplacement cen...
4,Lille,https://www.booking.com/hotel/fr/expressbyholi...,"Holiday Inn Express Lille Centre, an IHG Hotel","8,0",L'Holiday Inn Express Lille Centre est situé d...,"50.63075938,3.05808842",50.63075938,3.05808842,L'Holiday Inn Express Lille Centre est situé d...


In [31]:
lille.to_csv("Lille.csv")